# Querying Data from Google BigQuery to SQLite Database

In [1]:
import sqlite3
from google.cloud import bigquery
from google.api_core.exceptions import GoogleAPIError
import os
import polars as pl
import pandas as pd
import random

client = bigquery.Client(project="wedge-project-jbangtson")

In [4]:

# Function to run a query
def run_query(query):
    try:
        query_job = client.query(query)
        df = query_job.to_dataframe()
        #results = query_job.result()

        bytes_processed = query_job.total_bytes_processed
        mb_processed = bytes_processed / (1024 ** 2)
        cost_per_tb = 5.0


        tb_processed = bytes_processed / (1024 ** 4)  # Convert bytes to terabytes
        estimated_cost = tb_processed * cost_per_tb

        # Display the processed data and estimated cost
        print(f"Data processed: {mb_processed:.2f} MB")
        

        print(f"Estimated bytes processed: {bytes_processed}")
        print(f"Estimated cost: ${estimated_cost:.10f}\n\n")

        print(f"Estimated bytes processed against a full year of data: {bytes_processed*50}")
        print(f"Estimated cost against a full year of data: ${estimated_cost*50:.20f}")
        print(f"Estimated cost against a full year of data every 6 hours: ${(estimated_cost*50)*(4*365):.20f}\n\n---------------")

        

        return df.astype(str)
    except GoogleAPIError as e:
        print(f"Error running query: {e}")
        return None


## Summary Table One: Sales by Date by Hour
 
Sales by date by hour: By calendar date (YYYY-MM-DD) and hour of the day, 
determine the total spend in the store, the number of transactions, and a count of the number of items

In [5]:

sales_date_query = f"""
        SELECT
        FORMAT_DATETIME('%Y-%m-%d', datetime) AS date,
        EXTRACT(HOUR FROM datetime) AS hour,
        COUNT(DISTINCT datetime) AS transaction_count,
        SUM(quantity) AS total_items_sold,
        SUM(total) AS total_spend
        FROM
        `the_wedge_dataset.transArchive_*`
        GROUP BY
        date, hour
        ORDER BY
        date, hour;
        """

sales_date_df = run_query(sales_date_query)
sales_date_df.to_csv(f'E:\College\Fall 2024\ADA\Wedge\Wedge_Project\summary_tables\comma-sep\sales_date_summary.txt', sep='\t', index=False)

<>:17: SyntaxWarning: invalid escape sequence '\C'
<>:17: SyntaxWarning: invalid escape sequence '\C'
C:\Users\justi\AppData\Local\Temp\ipykernel_16164\3505907786.py:17: SyntaxWarning: invalid escape sequence '\C'
  sales_date_df.to_csv(f'E:\College\Fall 2024\ADA\Wedge\Wedge_Project\summary_tables\comma-sep\sales_date_summary.txt', sep='\t', index=False)
c:\Users\justi\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Data processed: 1896.51 MB
Estimated bytes processed: 1988637624
Estimated cost: $0.0090432769


Estimated bytes processed against a full year of data: 99431881200
Estimated cost against a full year of data: $0.45216384569357614964
Estimated cost against a full year of data every 6 hours: $660.15921471262117847800

---------------



## Summary Table Two: Sales by Owner by Year by Month
A file that has the following columns: card_no, year, month, sales, transactions, and items.


In [6]:

sales_owner_year = f"""
        SELECT
        card_no,
        EXTRACT(YEAR FROM datetime) AS year,
        EXTRACT(MONTH FROM datetime) AS month,
        COUNT(DISTINCT datetime) AS transactions,
        SUM(quantity) AS items,
        SUM(total) AS sales
        FROM
        `the_wedge_dataset.transArchive_*`
        WHERE
        card_no IS NOT NULL 
        GROUP BY
        card_no, year, month
        ORDER BY
        card_no, year, month;
        """

sales_owner_df = run_query(sales_owner_year)
sales_owner_df.to_csv(f'E:\College\Fall 2024\ADA\Wedge\Wedge_Project\summary_tables\comma-sep\sales_owner_summary.txt', sep='\t', index=False)

<>:20: SyntaxWarning: invalid escape sequence '\C'
<>:20: SyntaxWarning: invalid escape sequence '\C'
C:\Users\justi\AppData\Local\Temp\ipykernel_16164\739534758.py:20: SyntaxWarning: invalid escape sequence '\C'
  sales_owner_df.to_csv(f'E:\College\Fall 2024\ADA\Wedge\Wedge_Project\summary_tables\comma-sep\sales_owner_summary.txt', sep='\t', index=False)
c:\Users\justi\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Data processed: 2530.61 MB
Estimated bytes processed: 2653539520
Estimated cost: $0.0120669007


Estimated bytes processed against a full year of data: 132676976000
Estimated cost against a full year of data: $0.60334503359626978636
Estimated cost against a full year of data every 6 hours: $880.88374905055388808250

---------------



## Summary Table Three: Sales by Product Description by Year by Month
A file that has the following columns: upc, description, department number, department name, year, month, sales, transactions, and items.


In [7]:

sales_product_desc_year = f"""
        SELECT
    upc,
    description,
    department,
    CASE
        WHEN department = 1 THEN 'PACKAGED GROCERY'
        WHEN department = 2 THEN 'PRODUCE'
        WHEN department = 3 THEN 'BULK'
        WHEN department = 4 THEN 'REF GROCERY'
        WHEN department = 5 THEN 'CHEESE'
        WHEN department = 6 THEN 'FROZEN'
        WHEN department = 7 THEN 'BREAD'
        WHEN department = 8 THEN 'DELI'
        WHEN department = 9 THEN 'GEN MERCH'
        WHEN department = 10 THEN 'SUPPLEMENTS'
        WHEN department = 11 THEN 'PERSONAL CARE'
        WHEN department = 12 THEN 'HERBS&SPICES'
        WHEN department = 13 THEN 'MEAT'
        WHEN department = 14 THEN 'JUICE BAR'
        WHEN department = 15 THEN 'MISC P/I'
        WHEN department = 16 THEN 'FISH&SEAFOOD'
        WHEN department = 17 THEN 'BAKEHOUSE'
        WHEN department = 18 THEN 'FLOWERS'
        WHEN department = 19 THEN 'WEDGEWORLDWIDE'
        WHEN department = 20 THEN 'MISC P/I - WWW'
        WHEN department = 21 THEN 'CATERING'
        WHEN department = 22 THEN 'BEER & WINE'
        ELSE 'UNKNOWN'
    END AS department_name,
    EXTRACT(YEAR FROM datetime) AS year,
    EXTRACT(MONTH FROM datetime) AS month,
    SUM(total) AS sales,
    COUNT(DISTINCT datetime) AS transactions,
    SUM(quantity) AS items
FROM
  `the_wedge_dataset.transArchive_*`
GROUP BY
    description,
    year,
    month,
    department,
    upc
ORDER BY
    year,
    month,
    upc;
        """

sales_product_desc_year_df = run_query(sales_product_desc_year)
sales_product_desc_year_df.to_csv(f'E:\College\Fall 2024\ADA\Wedge\Wedge_Project\summary_tables\comma-sep\sales_product_desc_year.txt', sep='\t', index=False)

<>:51: SyntaxWarning: invalid escape sequence '\C'
<>:51: SyntaxWarning: invalid escape sequence '\C'
C:\Users\justi\AppData\Local\Temp\ipykernel_16164\1787018039.py:51: SyntaxWarning: invalid escape sequence '\C'
  sales_product_desc_year_df.to_csv(f'E:\College\Fall 2024\ADA\Wedge\Wedge_Project\summary_tables\comma-sep\sales_product_desc_year.txt', sep='\t', index=False)
c:\Users\justi\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Data processed: 5627.87 MB
Estimated bytes processed: 5901248863
Estimated cost: $0.0268357729


Estimated bytes processed against a full year of data: 295062443150
Estimated cost against a full year of data: $1.34178864368550421204
Estimated cost against a full year of data every 6 hours: $1959.01141978083614958450

---------------


## Creating Database and Tables


### Sales Date Summary

In [9]:
# Create a connection to the database file
conn = sqlite3.connect('E:\College\Fall 2024\ADA\Wedge\Wedge_Project\summary_tables\dbLite\wedge.db')

# Create a cursor object to interact with the database
cursor = conn.cursor()

# Create a table
cursor.execute('''
CREATE TABLE IF NOT EXISTS sales_date_summary (
date DATE,
hour INTEGER,
transaction_count INTEGER,
total_items_sold INTEGER,
total_spend REAL
)
''')

# Commit the changes and close the connection
conn.commit()
conn.close()

<>:2: SyntaxWarning: invalid escape sequence '\C'
<>:2: SyntaxWarning: invalid escape sequence '\C'
C:\Users\justi\AppData\Local\Temp\ipykernel_16164\3588922480.py:2: SyntaxWarning: invalid escape sequence '\C'
  conn = sqlite3.connect('E:\College\Fall 2024\ADA\Wedge\Wedge_Project\summary_tables\dbLite\wedge.db')


### Sales Owner Summary

In [11]:
# Create a connection to the database file
conn = sqlite3.connect('E:\College\Fall 2024\ADA\Wedge\Wedge_Project\summary_tables\dbLite\wedge.db')

# Create a cursor object to interact with the database
cursor = conn.cursor()

# Create a table
cursor.execute('''
CREATE TABLE IF NOT EXISTS sales_owner_summary (
card_no REAL,
year INTEGER,
month INTEGER,
transactions REAL,
items REAL,
sales REAL           
)
''')

# Commit the changes and close the connection
conn.commit()
conn.close()

<>:2: SyntaxWarning: invalid escape sequence '\C'
<>:2: SyntaxWarning: invalid escape sequence '\C'
C:\Users\justi\AppData\Local\Temp\ipykernel_16164\2865754892.py:2: SyntaxWarning: invalid escape sequence '\C'
  conn = sqlite3.connect('E:\College\Fall 2024\ADA\Wedge\Wedge_Project\summary_tables\dbLite\wedge.db')


### Sales Product Description


In [14]:
# Create a connection to the database file
conn = sqlite3.connect('E:\College\Fall 2024\ADA\Wedge\Wedge_Project\summary_tables\dbLite\wedge.db')

# Create a cursor object to interact with the database
cursor = conn.cursor()

# Create a table
cursor.execute('''
CREATE TABLE IF NOT EXISTS sales_product_desc_year_summary (
    upc REAL,
    description TEXT,
    department INTEGER,
    department_name TEXT,
    year INTEGER,
    month INTEGER,
    sales REAL,
    transactions INTEGER,
    items REAL
)
''')

# Commit the changes and close the connection
conn.commit()
conn.close()

<>:2: SyntaxWarning: invalid escape sequence '\C'
<>:2: SyntaxWarning: invalid escape sequence '\C'
C:\Users\justi\AppData\Local\Temp\ipykernel_16164\1719469209.py:2: SyntaxWarning: invalid escape sequence '\C'
  conn = sqlite3.connect('E:\College\Fall 2024\ADA\Wedge\Wedge_Project\summary_tables\dbLite\wedge.db')


## Uploading Data to SQLite

In [10]:


# Connect to the SQLite database
conn = sqlite3.connect('E:\College\Fall 2024\ADA\Wedge\Wedge_Project\summary_tables\dbLite\wedge.db')

# Upload the DataFrame to the SQLite table
sales_date_df.to_sql('sales_date_summary', conn, if_exists='append', index=False)

# Close the connection
conn.close()

print("Data uploaded successfully.")

Data uploaded successfully.


<>:2: SyntaxWarning: invalid escape sequence '\C'
<>:2: SyntaxWarning: invalid escape sequence '\C'
C:\Users\justi\AppData\Local\Temp\ipykernel_16164\2729933576.py:2: SyntaxWarning: invalid escape sequence '\C'
  conn = sqlite3.connect('E:\College\Fall 2024\ADA\Wedge\Wedge_Project\summary_tables\dbLite\wedge.db')


In [13]:


# Connect to the SQLite database
conn = sqlite3.connect('E:\College\Fall 2024\ADA\Wedge\Wedge_Project\summary_tables\dbLite\wedge.db')

# Upload the DataFrame to the SQLite table
sales_owner_df.to_sql('sales_owner_summary', conn, if_exists='append', index=False)

# Close the connection
conn.close()

print("Data uploaded successfully.")

<>:2: SyntaxWarning: invalid escape sequence '\C'
<>:2: SyntaxWarning: invalid escape sequence '\C'
C:\Users\justi\AppData\Local\Temp\ipykernel_16164\1883380411.py:2: SyntaxWarning: invalid escape sequence '\C'
  conn = sqlite3.connect('E:\College\Fall 2024\ADA\Wedge\Wedge_Project\summary_tables\dbLite\wedge.db')


Data uploaded successfully.


In [15]:


# Connect to the SQLite database
conn = sqlite3.connect('E:\College\Fall 2024\ADA\Wedge\Wedge_Project\summary_tables\dbLite\wedge.db')

# Upload the DataFrame to the SQLite table
sales_product_desc_year_df.to_sql('sales_product_desc_year_summary', conn, if_exists='append', index=False)

# Close the connection
conn.close()

print("Data uploaded successfully.")

<>:2: SyntaxWarning: invalid escape sequence '\C'
<>:2: SyntaxWarning: invalid escape sequence '\C'
C:\Users\justi\AppData\Local\Temp\ipykernel_16164\2355263730.py:2: SyntaxWarning: invalid escape sequence '\C'
  conn = sqlite3.connect('E:\College\Fall 2024\ADA\Wedge\Wedge_Project\summary_tables\dbLite\wedge.db')


Data uploaded successfully.
